#### DNN 모델을 이용하여 동영상 파일에 얼굴에 사각형 및 confidense 표시를 추가하여 display하세요

In [5]:
# 필요모듈 import
import cv2
import numpy as np

In [6]:
# 얼굴인식 모델 경로 및 min_confidense 설정 (동영상 파일 경로 및 파일명 저장)
model_name = 'res10_300x300_ssd_iter_140000.caffemodel'
prototxt_name = 'deploy.prototxt.txt'
min_confidence = 0.5
file_name = 'video/obama_01.mp4'

In [7]:
# detectAndDisplay 함수 제작 
# 이미지 형식 변환 및 얼굴 인식 표시 추가 , confidense 표시 추가 & display)
def detectAndDisplay(frame):
    # 모델 생성
    model = cv2.dnn.readNetFromCaffe(prototxt_name, model_name)
    # 형식 변환
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
            (300, 300), (104.0, 177.0, 123.0))
    # 모델에 이미지 세트
    model.setInput(blob)
    # 얼굴인식 자료 추출
    detections = model.forward()
    # 추출한 자료 갯수 만큼 반복실행
    for i in range(0, detections.shape[2]):
        # 얼굴로 인식된 자료가 진짜 얼굴인지를 표시한 확율 추출
        confidence = detections[0, 0, i, 2]
        # 확율이 50%(0.5) 초과인 데이터에 대해서 해당위치에 네모, 확율 표시
        if confidence > min_confidence:
            # 이미지 세로 가로 크기 추출
            (height, width) = frame.shape[:2]
            # 얼굴 위치의 좌표(0.0~1.0 사아의 상대적 좌표)와 세로 가로 길이를 곱해서
            # 얼굴 위치의 좌료를  픽셀 값으로 계산
            box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
            # 최종 정수로 변환해서 좌축 위 꼭지점과 우측아래 꼭지점 좌표로 
            # 사각형의 위치 확정
            (startX, startY, endX, endY) = box.astype("int")
            # 표시한 확율 제작
            text = "{:.2f}%".format(confidence * 100)
            # 확율 표시할 위치 선정
            # 표시할 위아래 위치가 이미지를 벗어나는 경우  y좌표에 +10 위치로 조정
            # 이미지를 벋어 나지 않는 경우는 얼굴 사각형의 바로 위(startY-10)로 설정
            y = startY - 10 if startY - 10 > 10 else startY + 10
            # 사각형 표시
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            # 확율 표시
            cv2.putText(frame,text,(startX,y),cv2.FONT_HERSHEY_SIMPLEX,0.5, \
                                                             (0, 255, 0), 1)
    #  완성된 이미지 화면 표시
    cv2.imshow("Face Detection by dnn", frame)  

In [8]:
# 동영상 캡쳐 화면을 이용한 detectAndDisplay 함수 호출
cap = cv2.VideoCapture(file_name)
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)
while True:
    ret, frame = cap.read()
    if frame is None:
        print('--(!) No captured frame -- Break!')
        break
    detectAndDisplay(frame)
    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()